# Autocorrelation
This script generates plots for the autocorrelation function and second-order statistics to evaluate the convergence of results.  

**Required Files:**  
1. **Savepoints**: These contain the coordinates of the point probes.  
2. **Flow* * Files*: These files include data recorded at each probe and must be stored in the "Input-files" folder.  

Ensure all necessary files are correctly placed before running the script.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16  # Set global font size

# Load savepoints
savepoints = np.loadtxt('savepoints')
dt = 0.0005
lp = len(savepoints)
path = "Input-files/"
os.makedirs(path, exist_ok=True)

# Initialize variables
u_mean = np.zeros(lp)
v_mean = np.zeros(lp)
w_mean = np.zeros(lp)

# Preallocate lists for variables
uf, vf, wf = [], [], []
uuf, vvf, wwf = [], [], []
uuf_mean, vvf_mean, wwf_mean = [], [], []
TKE = []
ACF = []
Tl = np.zeros(lp)

for i in range(lp):
    # Load file
    filename = f"Flow0_{savepoints[i, 0]:.2e}_{savepoints[i, 1]:.2e}_{savepoints[i, 2]:.2e}_dt_{dt:.4f}.dat"
    filename = os.path.join(path, filename)
    f = np.loadtxt(filename)

    # Delete unused data
    f = np.delete(f, np.s_[1:5], axis=1)
    f = np.delete(f, np.s_[4:7], axis=1)

    # Delete repeated data
    j = 1
    while j < len(f):
        if f[j, 0] <= f[j - 1, 0]:
            f = np.delete(f, j, axis=0)
        else:
            j += 1

    # Initialize variables for each savepoint
    if i == 0:
        uf = np.zeros((len(f), lp))
        vf = np.zeros((len(f), lp))
        wf = np.zeros((len(f), lp))
        uuf = np.zeros((len(f), lp))
        vvf = np.zeros((len(f), lp))
        wwf = np.zeros((len(f), lp))
        uuf_mean = np.zeros((len(f), lp))
        vvf_mean = np.zeros((len(f), lp))
        wwf_mean = np.zeros((len(f), lp))
        TKE = np.zeros((len(f), lp))
        ACF = np.zeros((4000, lp + 1))

    # Calculate variables
    u_mean[i] = np.mean(f[:, 1])
    v_mean[i] = np.mean(f[:, 2])
    w_mean[i] = np.mean(f[:, 3])
    uf[:, i] = f[:, 1] - u_mean[i]
    vf[:, i] = f[:, 2] - v_mean[i]
    wf[:, i] = f[:, 3] - w_mean[i]
    uuf[:, i] = uf[:, i] ** 2
    vvf[:, i] = vf[:, i] ** 2
    wwf[:, i] = wf[:, i] ** 2

    for j in range(len(f)):
        uuf_mean[j, i] = np.mean(uuf[:j + 1, i])
        vvf_mean[j, i] = np.mean(vvf[:j + 1, i])
        wwf_mean[j, i] = np.mean(wwf[:j + 1, i])

    TKE[:, i] = (uuf_mean[:, i] + vvf_mean[:, i] + wwf_mean[:, i]) / 2

    # Autocorrelation functions
    s0 = np.mean(uuf[:, i])
    for lag in range(1, 4000):
        sk = np.sum(uf[:-lag, i] * uf[lag:, i]) / len(uf[:-lag])
        ACF[lag, i + 1] = sk / s0
        if i == 0:
            ACF[lag, 0] = dt * lag

    # Integral time scale
    sum_acf = 0
    for n in range(1, len(ACF)):
        sum_acf += (ACF[n, i + 1] + ACF[n - 1, i + 1]) / 2 * dt
        if ACF[n, i + 1] <= 0:
            break
    Tl[i] = sum_acf

# Final calculations
Ts = Tl * 200

# Plotting

# Plot u'u' mean
time = np.arange(0, len(f) * dt, dt)
plt.figure(figsize=(12, 6))
plt.plot(time, uuf_mean)
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("r$\overline{u'u'}$")
plt.title("Mean u'u'")
plt.savefig(os.path.join(path, "ReS.tif"))
plt.show()

# Plot TKE
plt.figure(figsize=(12, 6))
plt.plot(time, TKE)
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("TKE")
plt.title("TKE")
plt.savefig(os.path.join(path, "TKE.tif"))
plt.show()

# Plot ACF
plt.figure(figsize=(12, 6))
plt.plot(ACF[:, 0], ACF[:, 1:])
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Autocorrelation Function")
plt.title("ACF")
plt.savefig(os.path.join(path, "ACF.tif"))
plt.show()

# Save Integral Time Scale
with open(os.path.join(path, "IntegralTimeScale.txt"), "w") as fw:
    fw.write("Tl: ")
    fw.write(" ".join(map(str, Tl)))
    fw.write("\n")
    fw.write("Ts=200ac0*Tl: ")
    fw.write(" ".join(map(str, Ts)))


Updated, with black color and different line styles.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Set the font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16  # Set global font size

# Load savepoints
savepoints = np.loadtxt('savepoints')
dt = 0.0005
lp = len(savepoints)
path = "Input-files/"
os.makedirs(path, exist_ok=True)

# Initialize variables
u_mean = np.zeros(lp)
v_mean = np.zeros(lp)
w_mean = np.zeros(lp)

# Preallocate lists for variables
uf, vf, wf = [], [], []
uuf, vvf, wwf = [], [], []
uuf_mean, vvf_mean, wwf_mean = [], [], []
TKE = []
ACF = []
Tl = np.zeros(lp)

for i in range(lp):
    # Load file
    filename = f"Flow0_{savepoints[i, 0]:.2e}_{savepoints[i, 1]:.2e}_{savepoints[i, 2]:.2e}_dt_{dt:.4f}.dat"
    filename = os.path.join(path, filename)
    f = np.loadtxt(filename)

    # Delete unused data
    f = np.delete(f, np.s_[1:5], axis=1)
    f = np.delete(f, np.s_[4:7], axis=1)

    # Delete repeated data
    j = 1
    while j < len(f):
        if f[j, 0] <= f[j - 1, 0]:
            f = np.delete(f, j, axis=0)
        else:
            j += 1

    # Initialize variables for each savepoint
    if i == 0:
        uf = np.zeros((len(f), lp))
        vf = np.zeros((len(f), lp))
        wf = np.zeros((len(f), lp))
        uuf = np.zeros((len(f), lp))
        vvf = np.zeros((len(f), lp))
        wwf = np.zeros((len(f), lp))
        uuf_mean = np.zeros((len(f), lp))
        vvf_mean = np.zeros((len(f), lp))
        wwf_mean = np.zeros((len(f), lp))
        TKE = np.zeros((len(f), lp))
        ACF = np.zeros((4000, lp + 1))

    # Calculate variables
    u_mean[i] = np.mean(f[:, 1])
    v_mean[i] = np.mean(f[:, 2])
    w_mean[i] = np.mean(f[:, 3])
    uf[:, i] = f[:, 1] - u_mean[i]
    vf[:, i] = f[:, 2] - v_mean[i]
    wf[:, i] = f[:, 3] - w_mean[i]
    uuf[:, i] = uf[:, i] ** 2
    vvf[:, i] = vf[:, i] ** 2
    wwf[:, i] = wf[:, i] ** 2

    for j in range(len(f)):
        uuf_mean[j, i] = np.mean(uuf[:j + 1, i])
        vvf_mean[j, i] = np.mean(vvf[:j + 1, i])
        wwf_mean[j, i] = np.mean(wwf[:j + 1, i])

    TKE[:, i] = (uuf_mean[:, i] + vvf_mean[:, i] + wwf_mean[:, i]) / 2

    # Autocorrelation functions
    s0 = np.mean(uuf[:, i])
    for lag in range(1, 4000):
        sk = np.sum(uf[:-lag, i] * uf[lag:, i]) / len(uf[:-lag])
        ACF[lag, i + 1] = sk / s0
        if i == 0:
            ACF[lag, 0] = dt * lag

    # Integral time scale
    sum_acf = 0
    for n in range(1, len(ACF)):
        sum_acf += (ACF[n, i + 1] + ACF[n - 1, i + 1]) / 2 * dt
        if ACF[n, i + 1] <= 0:
            break
    Tl[i] = sum_acf

# Final calculations
Ts = Tl * 200

# Plotting

linestyles = ['-', '--', ':', '-.']  # Define line styles

# Plot u'u' mean
plt.figure(figsize=(12, 6))
for i in range(lp):
    plt.plot(time, uuf_mean[:, i], color='black', linestyle=linestyles[i % len(linestyles)])
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel(r"$\overline{u'u'}$")
plt.title("Mean u'u'")
plt.savefig(os.path.join(path, "ReS.tif"))
plt.show()

# Plot TKE
time = np.arange(0, len(f) * dt, dt)
plt.figure(figsize=(12, 6))
for i in range(lp):
    plt.plot(time, TKE[:, i], color='black', linestyle=linestyles[i % len(linestyles)])
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("TKE")
plt.title("TKE")
plt.savefig(os.path.join(path, "TKE.tif"))
plt.show()

# Plot ACF
plt.figure(figsize=(12, 6))
for i in range(lp):
    plt.plot(ACF[:, 0], ACF[:, i + 1], color='black', linestyle=linestyles[i % len(linestyles)])
plt.legend([f'P{i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Autocorrelation Function")
plt.title("ACF")
plt.savefig(os.path.join(path, "ACF.tif"))
plt.show()

# Save Integral Time Scale
with open(os.path.join(path, "IntegralTimeScale.txt"), "w") as fw:
    fw.write("Tl: ")
    fw.write(" ".join(map(str, Tl)))
    fw.write("\n")
    fw.write("Ts=200ac0*Tl: ")
    fw.write(" ".join(map(str, Ts)))


In case that the savepoints just contains one point, you will need the following script:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16  # Set global font size

# Load savepoints
savepoints = np.loadtxt('savepoints')
# Reshape if needed
if savepoints.ndim == 1:
    savepoints = savepoints.reshape(-1, 3)  # Ensure it is a 2D array with 3 columns

# Check the shape
print("Savepoints shape:", savepoints.shape)    

dt = 0.0005
lp = len(savepoints)
path = "Input-files/"
os.makedirs(path, exist_ok=True)

# Initialize variables
u_mean = np.zeros(lp)
v_mean = np.zeros(lp)
w_mean = np.zeros(lp)

# Preallocate lists for variables
uf, vf, wf = [], [], []
uuf, vvf, wwf = [], [], []
uuf_mean, vvf_mean, wwf_mean = [], [], []
TKE = []
ACF = []
Tl = np.zeros(lp)

for i in range(lp):
    # Load file
    filename = f"Flow0_{savepoints[i, 0]:.2e}_{savepoints[i, 1]:.2e}_{savepoints[i, 2]:.2e}_dt_{dt:.4f}.dat"
    filename = os.path.join(path, filename)
    f = np.loadtxt(filename)

    # Delete unused data
    f = np.delete(f, np.s_[1:5], axis=1)
    f = np.delete(f, np.s_[4:7], axis=1)

    # Delete repeated data
    j = 1
    while j < len(f):
        if f[j, 0] <= f[j - 1, 0]:
            f = np.delete(f, j, axis=0)
        else:
            j += 1

    # Initialize variables for each savepoint
    if i == 0:
        uf = np.zeros((len(f), lp))
        vf = np.zeros((len(f), lp))
        wf = np.zeros((len(f), lp))
        uuf = np.zeros((len(f), lp))
        vvf = np.zeros((len(f), lp))
        wwf = np.zeros((len(f), lp))
        uuf_mean = np.zeros((len(f), lp))
        vvf_mean = np.zeros((len(f), lp))
        wwf_mean = np.zeros((len(f), lp))
        TKE = np.zeros((len(f), lp))
        ACF = np.zeros((4000, lp + 1))

    # Calculate variables
    u_mean[i] = np.mean(f[:, 1])
    v_mean[i] = np.mean(f[:, 2])
    w_mean[i] = np.mean(f[:, 3])
    uf[:, i] = f[:, 1] - u_mean[i]
    vf[:, i] = f[:, 2] - v_mean[i]
    wf[:, i] = f[:, 3] - w_mean[i]
    uuf[:, i] = uf[:, i] ** 2
    vvf[:, i] = vf[:, i] ** 2
    wwf[:, i] = wf[:, i] ** 2

    for j in range(len(f)):
        uuf_mean[j, i] = np.mean(uuf[:j + 1, i])
        vvf_mean[j, i] = np.mean(vvf[:j + 1, i])
        wwf_mean[j, i] = np.mean(wwf[:j + 1, i])

    TKE[:, i] = (uuf_mean[:, i] + vvf_mean[:, i] + wwf_mean[:, i]) / 2

    # Autocorrelation functions
    s0 = np.mean(uuf[:, i])
    for lag in range(1, 4000):
        sk = np.sum(uf[:-lag, i] * uf[lag:, i]) / len(uf[:-lag])
        ACF[lag, i + 1] = sk / s0
        if i == 0:
            ACF[lag, 0] = dt * lag

    # Integral time scale
    sum_acf = 0
    for n in range(1, len(ACF)):
        sum_acf += (ACF[n, i + 1] + ACF[n - 1, i + 1]) / 2 * dt
        if ACF[n, i + 1] <= 0:
            break
    Tl[i] = sum_acf

# Final calculations
Ts = Tl * 300

# Plotting

# Plot u'u' mean
time = np.arange(0, len(f) * dt, dt)
plt.figure(figsize=(12, 6))
plt.plot(time, uuf_mean)
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("<u'u'>")
plt.title("Mean u'u'")
plt.savefig(os.path.join(path, "ReS.tif"))
plt.show()

# Plot TKE
plt.figure(figsize=(12, 6))
plt.plot(time, TKE)
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Turbulent Kinetic Energy")
plt.title("TKE")
plt.savefig(os.path.join(path, "TKE.tif"))
plt.show()

# Plot ACF
plt.figure(figsize=(12, 6))
plt.plot(ACF[:, 0], ACF[:, 1:])
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Autocorrelation Function")
plt.title("ACF")
plt.savefig(os.path.join(path, "ACF.tif"))
plt.show()

# Save Integral Time Scale
with open(os.path.join(path, "IntegralTimeScale.txt"), "w") as fw:
    fw.write("Tl: ")
    fw.write(" ".join(map(str, Tl)))
    fw.write("\n")
    fw.write("Ts=50ac0*Tl: ")
    fw.write(" ".join(map(str, Ts)))


This script is the same as previous one, but with slight change in reshaping. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16  # Set global font size

# Load savepoints
savepoints = np.loadtxt('savepoints')

# Reshape if needed
if savepoints.ndim == 1:
    savepoints = savepoints.reshape(1, -1)  # Ensure it's a 2D array with one row

# Check the shape
print("Savepoints shape:", savepoints.shape)

# Other code logic
dt = 0.0005
path = "Input-files/"
lp = len(savepoints)  # Number of savepoints

# Initialize variables
u_mean = np.zeros(lp)
v_mean = np.zeros(lp)
w_mean = np.zeros(lp)

# Preallocate lists for variables
uf, vf, wf = [], [], []
uuf, vvf, wwf = [], [], []
uuf_mean, vvf_mean, wwf_mean = [], [], []
TKE = []
ACF = []
Tl = np.zeros(lp)

for i in range(lp):
    # Load file for each savepoint
    filename = f"Flow0_{savepoints[i, 0]:.2e}_{savepoints[i, 1]:.2e}_{savepoints[i, 2]:.2e}_dt_{dt:.4f}.dat"
    filepath = os.path.join(path, filename)

    try:
        f = np.loadtxt(filepath)
        print(f"Processing file: {filepath}")
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        continue

    # Delete unused data
    f = np.delete(f, np.s_[1:5], axis=1)
    f = np.delete(f, np.s_[4:7], axis=1)

    # Delete repeated data
    j = 1
    while j < len(f):
        if f[j, 0] <= f[j - 1, 0]:
            f = np.delete(f, j, axis=0)
        else:
            j += 1

    # Initialize variables for each savepoint
    if i == 0:
        uf = np.zeros((len(f), lp))
        vf = np.zeros((len(f), lp))
        wf = np.zeros((len(f), lp))
        uuf = np.zeros((len(f), lp))
        vvf = np.zeros((len(f), lp))
        wwf = np.zeros((len(f), lp))
        uuf_mean = np.zeros((len(f), lp))
        vvf_mean = np.zeros((len(f), lp))
        wwf_mean = np.zeros((len(f), lp))
        TKE = np.zeros((len(f), lp))
        ACF = np.zeros((80000, lp + 1))

    # Calculate variables
    u_mean[i] = np.mean(f[:, 1])
    v_mean[i] = np.mean(f[:, 2])
    w_mean[i] = np.mean(f[:, 3])
    uf[:, i] = f[:, 1] - u_mean[i]
    vf[:, i] = f[:, 2] - v_mean[i]
    wf[:, i] = f[:, 3] - w_mean[i]
    uuf[:, i] = uf[:, i] ** 2
    vvf[:, i] = vf[:, i] ** 2
    wwf[:, i] = wf[:, i] ** 2

    for j in range(len(f)):
        uuf_mean[j, i] = np.mean(uuf[:j + 1, i])
        vvf_mean[j, i] = np.mean(vvf[:j + 1, i])
        wwf_mean[j, i] = np.mean(wwf[:j + 1, i])

    TKE[:, i] = (uuf_mean[:, i] + vvf_mean[:, i] + wwf_mean[:, i]) / 2

    # Autocorrelation functions
    s0 = np.mean(uuf[:, i])
    for lag in range(1, 80000):
        sk = np.sum(uf[:-lag, i] * uf[lag:, i]) / len(uf[:-lag])
        ACF[lag, i + 1] = sk / s0
        if i == 0:
            ACF[lag, 0] = dt * lag

    # Integral time scale
    sum_acf = 0
    for n in range(1, len(ACF)):
        sum_acf += (ACF[n, i + 1] + ACF[n - 1, i + 1]) / 2 * dt
        if ACF[n, i + 1] <= 0:
            break
    Tl[i] = sum_acf

# Final calculations
Ts = Tl * 300

# Plotting

# Plot u'u' mean
time = np.arange(0, len(f) * dt, dt)
plt.figure(figsize=(12, 6))
plt.plot(time, uuf_mean)
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("<u'u'>")
plt.title("Mean u'u'")
plt.savefig(os.path.join(path, "ReS.tif"))
plt.show()

# Plot TKE
plt.figure(figsize=(12, 6))
plt.plot(time, TKE)
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Turbulent Kinetic Energy")
plt.title("TKE")
plt.savefig(os.path.join(path, "TKE.tif"))
plt.show()

# Plot ACF
plt.figure(figsize=(12, 6))
plt.plot(ACF[:, 0], ACF[:, 1:])
plt.legend([f'Point {i + 1}' for i in range(lp)])
plt.xlabel("Time (s)")
plt.ylabel("Autocorrelation Function")
plt.title("ACF")
plt.savefig(os.path.join(path, "ACF.tif"))
plt.show()

# Save Integral Time Scale
with open(os.path.join(path, "IntegralTimeScale.txt"), "w") as fw:
    fw.write("Tl: ")
    fw.write(" ".join(map(str, Tl)))
    fw.write("\n")
    fw.write("Ts=300*Tl: ")
    fw.write(" ".join(map(str, Ts)))

At the end, try any of them with different savepoints and pick one script for final use.